Przykład DL - Klasyfikator cyfr
-------------------------------

Import bibliotek

In [1]:
from fastai.vision.all import *

Pobranie próbki zb. obrazów cyfr

In [2]:
path = untar_data(URLs.MNIST_SAMPLE)

In [3]:
path.ls()

(#3) [Path('C:/Users/Wojtek/.fastai/data/mnist_sample/labels.csv'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/valid')]

In [4]:
(path/'train').ls()

(#2) [Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/3'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/7')]

In [5]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
sevens

(#6265) [Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/7/10002.png'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/7/1001.png'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/7/10014.png'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/7/10019.png'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/7/10039.png'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/7/10046.png'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/7/10050.png'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/7/10063.png'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/7/10077.png'),Path('C:/Users/Wojtek/.fastai/data/mnist_sample/train/7/10086.png')...]

Przykładowe odpalenie obrazka

In [6]:
im3_path = threes[1]
im3 = Image.open(im3_path)
im3

Przykładowa konwersja obrazu do tablicy NumPy i tensora PyTorch

In [7]:
# NumPy
array(im3)[4:10,4:10]
# PyTorch tensor
tensor(im3)[4:10,4:10]

tensor([[  0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,  29],
        [  0,   0,   0,  48, 166, 224],
        [  0,  93, 244, 249, 253, 187],
        [  0, 107, 253, 253, 230,  48],
        [  0,   3,  20,  20,  15,   0]], dtype=torch.uint8)

In [8]:
# Wizualizacja
im3_t = tensor(im3)
df = pd.DataFrame(im3_t[4:10, 4:18])
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,29,150,195,254,255,254,176,193,150
2,0,0,0,48,166,224,253,253,234,196,253,253,253,253
3,0,93,244,249,253,187,46,10,8,4,10,194,253,253
4,0,107,253,253,230,48,0,0,0,0,0,192,253,253
5,0,3,20,20,15,0,0,0,0,0,43,224,253,245


Tworzenie modelu - official

In [9]:
# Zdefiniowanie tensorów do przechowywania wszystkich obrazow
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
#show_image(three_tensors[3]);


In [10]:
# Rzutowanie wynikowych tensorów na float
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255
stacked_sevens.shape

torch.Size([6265, 28, 28])

In [11]:
mean3 = stacked_threes.mean(0)
mean7 = stacked_sevens.mean(0)
#show_image(mean7);

In [12]:
a_3 = stacked_threes[5]

Obliczanie wielkości różnic między obrazem uśrednionym a dowolnym

In [13]:
# Średnia różnica bezwzględna - norma L1
dist_3_abs = (a_3 - mean3).abs().mean()
# Srednia z kwadratu różnic
dist_3_sqr = ((a_3 - mean3)**2).mean().sqrt()
print(dist_3_abs,dist_3_sqr)

dist_7_abs = (a_3 - mean7).abs().mean()
dist_7_sqr = ((a_3 - mean7)**2).mean().sqrt()
print(dist_7_abs,dist_7_sqr)

tensor(0.1700) tensor(0.3016)
tensor(0.2198) tensor(0.3829)


In [14]:
import torch.nn.functional as Ft

In [15]:
Ft.l1_loss(a_3.float(),mean7), Ft.mse_loss(a_3,mean7).sqrt()

(tensor(0.2198), tensor(0.3829))

In [20]:
# Definiowanie wskaznika poprawnosci predykcji
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/"valid"/'3').ls()])
valid_3_tens = valid_3_tens.float()/255

valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/"valid"/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
#valid_3_tens.shape, valid_7_tens.shape

(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [22]:
def mnist_distance(a,b):
    return (a-b).abs().mean((-1,-2))
#mnist_distance(a_3, mean3)


In [24]:
valid_3_dist = mnist_distance(valid_3_tens, mean3)
valid_3_dist, valid_3_dist.shape

(tensor([0.1280, 0.1623, 0.1242,  ..., 0.1508, 0.1263, 0.1260]),
 torch.Size([1010]))

In [27]:
def is_3(x):
    return mnist_distance(x, mean3) < mnist_distance(x, mean7)

is_3(valid_3_tens)

tensor([ True,  True,  True,  ..., False,  True,  True])

In [29]:
acc_3 = is_3(valid_3_tens).float().mean()
acc_7 = (1 - is_3(valid_7_tens).float()).mean()
acc_3, acc_7, (acc_3+acc_7)/2

(tensor(0.9168), tensor(0.9854), tensor(0.9511))